In [ ]:
import random

from pareto import utils
import pandas as pd
import numpy as np

names = utils.generate_toy_story_machine_names(count=100)


df = pd.DataFrame(names, columns=["machine_name"])
df["target_util_rate"] = random.choices([0.3, 0.4, 0.5], k=len(names))
df["total_hours_year"] = 8760
df["utilized_hours_year"] = np.random.normal(2028, 300, len(names)).astype(int)
df['utilization_rate'] = df['utilized_hours_year'] / df['total_hours_year']
df['utilization_rate_normalized'] = df['utilization_rate'] / df['target_util_rate']
df['under_utilized_rate_normalized'] = 1 - df['utilization_rate_normalized']


In [ ]:
df

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

VALUE_COL = "under_utilized_rate_normalized"
TITLE = "Pareto Analysis: Normalized Machine Utilization Rate"

df_pareto = df.sort_values(by=VALUE_COL, ascending=False).reset_index(drop=True)

# 2. Calculate the cumulative sum and cumulative percentage
total_sum = df_pareto[VALUE_COL].sum()
df_pareto["cumulative_sum"] = df_pareto[VALUE_COL].cumsum()
df_pareto["cumulative_percent"] = (df_pareto["cumulative_sum"] / total_sum) * 100

# 3. Create the Plotly figure with two y-axes
fig = make_subplots(specs=[[{"secondary_y": True}]])

# 4. Add the Bar Chart (Primary Y-axis)
fig.add_trace(
    go.Bar(
        x=df_pareto["machine_name"],
        y=df_pareto[VALUE_COL],
        name=VALUE_COL,
        marker_color="rgb(34, 139, 34)",  # Forest Green for resource usage
        hovertemplate="%{x}: %{y:.2f}<extra></extra>",
    ),
    secondary_y=False,
)

# 5. Add the Line Chart (Secondary Y-axis for Cumulative Percentage)
fig.add_trace(
    go.Scatter(
        x=df_pareto["machine_name"],
        y=df_pareto["cumulative_percent"],
        name="Cumulative Percentage",
        mode="lines+markers",
        line=dict(color="rgb(255, 69, 0)", width=3),  # Red/Orange for focus line
        marker=dict(symbol="circle", size=8),
        hovertemplate="%{x}: %{y:.1f}%%<extra></extra>",
    ),
    secondary_y=True,
)

# 6. Customize Layout
fig.update_layout(
    title={
        "text": TITLE,
        "y": 0.95,
        "x": 0.5,
        "xanchor": "center",
        "yanchor": "top",
        "font": dict(size=20, family="Inter, sans-serif"),
    },
    xaxis_title="Machine Name (Sorted by Normalized Rate)",
    margin=dict(l=40, r=40, t=60, b=40),
    template="plotly_white",
)

# Set y-axes titles and ranges
fig.update_yaxes(
    title_text=f"{VALUE_COL}",
    secondary_y=False,
    range=[0, df_pareto[VALUE_COL].max() * 1.05],  # Slight buffer
    showgrid=False,
)
fig.update_yaxes(
    title_text="Cumulative Percentage (%)",
    secondary_y=True,
    range=[0, 100],
    tickvals=list(range(0, 101, 10)),
    gridcolor="lightgray",
)

# Add 80/20 line
fig.add_hline(
    y=80,
    line_dash="dash",
    line_color="red",
    secondary_y=True,
    annotation_text="80% Threshold",
    annotation_position="top right",
    annotation_font_size=12,
    annotation_font_color="red",
)

# Hide x-axis tick labels if there are too many (100 labels look messy)
if len(df_pareto) > 50:
    fig.update_xaxes(showticklabels=False)

fig.show()
